In [1]:

import json


from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate



def parent_child(input_filename):
    with open(input_filename) as f:
        data = json.load(f)
    parent_docs=[]
    child_docs=[]
    for chapter in data:
        for section in chapter['sections']:
            parent_doc = Document(
                page_content=section['original_content'],
                metadata={
                    'section_id': section["section_id"],
                    'title': section["title"],
                    'chapter':chapter['chapter_name'],
                    'doc_type': "parent"
                }
            )
            parent_docs.append(parent_doc)
            for unit in section['atomic_units']:
                child_doc = Document(
                    page_content= unit['enriched_context'],
                    metadata={
                        'parent_section_id': unit['parent_section_id'],
                        'unit_type': unit['unit_type'],
                        'chunk_index': unit['chunk_index'],
                        'doc_type': "child"
                    }
                )
                child_docs.append(child_doc)
    print(f"✅ Successfully prepared data.")
    print(f"   - Parent Documents (Full Sections): {len(parent_docs)}")
    print(f"   - Child Documents (Searchable Units): {len(child_docs)}")
    return parent_docs, child_docs

parents, children = parent_child("cpa_anchored_refined_v2.json")





/Users/srichandrasamanapalli/Who to sue next/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Successfully prepared data.
   - Parent Documents (Full Sections): 107
   - Child Documents (Searchable Units): 572


In [2]:
import gc
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import os
import shutil
from huggingface_hub import snapshot_download
from tqdm.notebook import tqdm
if 'vector_db' in globals():
    del vector_db

gc.collect()
CHROMA_PATH = "./chroma_db_store_new"
MODEL_NAME = "BAAI/bge-m3"
if os.path.exists(CHROMA_PATH):
     shutil.rmtree(CHROMA_PATH)
snapshot_download(repo_id=MODEL_NAME,repo_type='model')

embeddings=HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

vector_db=Chroma.from_documents(
    embedding =embeddings,
    documents=children,
    persist_directory=CHROMA_PATH,
    collection_name='cpa_legal_index'

)




Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 140121.51it/s]


In [3]:
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
bm25retriever=BM25Retriever.from_documents(children)
bm25retriever.k=5



In [4]:
chroma_retriever = vector_db.as_retriever(search_kwargs={"k":5})

ensembleretriever=EnsembleRetriever(retrievers=[bm25retriever, chroma_retriever],weights=[0.5,0.5])

In [6]:
import pickle


with open('bm25_retriever.pkl', 'wb') as f:
    pickle.dump(bm25retriever, f)



bm25_retriever.pkl created successfully!


In [5]:
with open('cpa_anchored_refined_v2.json','r',encoding='UTF8') as f:
    data=json.load(f)

parent_store={}
for chapter in data:
    for section in chapter['sections']:
        s_id=section['section_id']
        parent_store[s_id]=section['original_content']
user_query=input('Enter you query:')
results=ensembleretriever.invoke(user_query)
final_context=[]
seen_ids=set()
for child in results:
    if child.metadata.get('parent_section_id') not in seen_ids:
        final_context.append(parent_store[child.metadata.get('parent_section_id')])
        seen_ids.add(child.metadata.get('parent_section_id'))
print(seen_ids)
print(final_context)

KeyboardInterrupt: Interrupted by user

In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

llm=ChatGoogleGenerativeAI(temperature=0.3,max_retries=3,model="gemini-2.5-flash")

In [ ]:
template='''
You are an expert lawyer specialised in indian consumer protection law.
I want you to answer my questions based on only the following sections from the consumer law:
{context}

My question is {question}
your answer should follow the follow
'''

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt= ChatPromptTemplate.from_template(template)


In [ ]:
chain = prompt | llm | StrOutputParser()

In [ ]:
formatted_data="\n\n".join(final_context)

In [ ]:
response= chain.invoke({"context":formatted_data, "question": user_query})

In [ ]:
response
